In [145]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Georgia

## Election Results Documentation:

Election results from the Georgia Secretary of State Elections Division (https://sos.ga.gov/index.php/Elections/current_and_past_elections_results). Presidential recount results from the Georgia Secretary of State Elections Division via Reuters.

## Shapefile Source Documentation:

Precinct shapefile primarily from the Georgia General Assembly Reapportionment Office (http://www.legis.ga.gov/Joint/reapportionment/en-US/default.aspx).  
Cobb, DeKalb, and Gwinnett counties instead use shapefiles from the U.S. Census Bureau's 2020 Redistricting Data Program.  
Forsyth and Fulton use shapefiles sourced from the respective counties.

## Shapefile Modifications Documentation:

Three of the four VTDs in Chattahoochee County are comprised of Fort Benning. However, the county only reports one polling location for all voters, including residents of Fort Benning that vote within the county. The four Chattahoochee County VTDs have therefore been merged in the shapefile.

The following additional modifications reflect changes made prior to the 2020 general election.

Barrow: Merge 2/15, 3/12, 4/14, 5/7, 6/10/13, 8/9, 11/16; Adjust new 2/13 boundary  
Bartow: Split Cassville/Hamilton Crossing  
Candler: Merge Candler/Metter as Jack Strickland Comm Center  
Chatham: Split 7-7/8-16, 7-12/7-16; Realign 7-06C/7-07C  
Chatooga: Split Cloudland/Teloga along ridgeline that marks boundary between them with the USGS Topographic Contour shapefile  
Clayton: Split Ellenswood 1/2, Jonesboro 1/17/19, Lovejoy 3/6/7, Morrow 3/11, 5/10, Oak 3/5   
Cobb: Split Bells Ferry 3/4, Dobbins 1/2, Marietta 3A/3B, Smyrna 3A/3B  
Columbia: Split Bessie Thomas/2nd Mt Moriah, Harlem Branch/Harlem Senior Ctr; Merge Blanchard Park/MTZ Col FD;  Align multiple precincts with county maps  
Coweta: Merge Arts Centre/Jefferson Parkway as Newnan Centre  
Fulton: Merge CP07A/CP07D, CH01/CH04B, SS29A/SS29B, UC031/UC035  
DeKalb: Split Clarkston/Clarkston Comm Ctr; Realign Decatur/Oakhurst; Align precincts with Atlanta, Brookhaven, Decatur, Tucker city limits   
Gwinnett: Adjust Baycreek F/G, Berkshire J/M, Cates D/F, Garners C/B, Lawrenceville G/N, Pinckneyville S/T, Rockbridge A/G  
Lowndes: Split Northgate Assembly/Trinity, Jaycee/Mt Calvary/Northside/VSU  
Oconee: Merge Annex/City Hall; Align City Hall with Watkinsville city limits  
Paulding: Reorganize 12 precincts into 19 precincts as redrawn in 2019  
Randolph: Merge Carnegie/Cuthbert-Courthouse, 4th District/Fountain Bridge/Shellman  
Troup: Split Mountville between Gardner Newman/Hogansville/Rosemont; Align multiple precincts with county maps  
Towns: Merge Macedonia/Tate City  
Wilkes: Align 1/2A boundary with the voter file    
Note that the leading zeros in the Paulding County precinct IDs are included in some election reports and omitted in others. The shapefile includes the leading zeros consistent with the voter file.

## Candidate List

G20PRERTRU - Donald J. Trump (Republican Party)  
G20PREDBID - Joseph R. Biden (Democratic Party)  
G20PRELJOR - Jo Jorgensen (Libertarian Party)  
  
C20PRERTRU - Donald J. Trump (Republican Party)  
C20PREDBID - Joseph R. Biden (Democratic Party)  
C20PRELJOR - Jo Jorgensen (Libertarian Party)  
  
G20USSRPER - David A. Perdue (Republican Party)  
G20USSDOSS - Jon Ossoff (Democratic Party)  
G20USSLHAZ - Shane Hazel (Libertarian Party)  
  
S20USSRLOE - Kelly Loeffler (Republican Party)  
S20USSRCOL - Doug Collins (Republican Party)  
S20USSRGRA - Derrick E. Grayson (Republican Party)  
S20USSRJAC - Annette Davis Jackson (Republican Party)  
S20USSRTAY - Kandiss Taylor (Republican Party)  
S20USSRJOH - A. Wayne Johnson (Republican Party)  
S20USSDWAR - Raphael Warnock (Democratic Party)  
S20USSDJAC - Deborah Jackson (Democratic Party)  
S20USSDLIE - Matt Lieberman (Democratic Party)  
S20USSDJOH - Tamara Johnson-Shealey (Democratic Party)  
S20USSDJAM - Jamesia James (Democratic Party)  
S20USSDSLA - Joy Felicia Slade (Democratic Party)  
S20USSDWIN - Richard Dien Winfield (Democratic Party)  
S20USSDTAR - Ed Tarver (Democratic Party)  
S20USSLSLO - Brian Slowinski (Libertarian Party)  
S20USSGFOR - John Fortuin (Green Party)  
S20USSIBUC - Allen Buckley (Independent)  
S20USSIBAR - Al Bartell (Independent)  
S20USSISTO - Valencia Stovall (Independent)  
S20USSIGRE - Michael Todd Greene (Independent)  
  
G20PSCRSHA - Jason Shaw (Republican Party)  
G20PSCDBRY - Robert G. Bryant (Democratic Party)  
G20PSCLMEL - Elizabeth Melton (Libertarian Party)  
  
G20PSCRMCD - Lauren Bubba McDonald, Jr. (Republican Party)  
G20PSCDBLA - Daniel Blackman (Democratic Party)  
G20PSCLWIL - Nathan Wilson (Libertarian Party)  

R21USSRPER - David A. Perdue (Republican Party)  
R21USSDOSS - Jon Ossoff (Democratic Party)  

R21USSRLOE - Kelly Loeffler (Republican Party)  
R21USSDWAR - Raphael Warnock (Democratic Party)  

R21PSCRMCD - Lauren Bubba McDonald, Jr. (Republican Party)  
P21PSCDBLA - Daniel Blackman (Democratic Party)  

### Load the VEST file

In [92]:
vest_ga_20 = gp.read_file("./raw-from-source/VEST/ga_2020/ga_2020.shp")
print(vest_ga_20.shape)

(2679, 50)


In [93]:
# Take a look
vest_ga_20.head(1)

#Create a list of the race columns
data_columns = [i for i in vest_ga_20.columns if (("20" in i) | ("21" in i))]
print(data_columns)

['G20PRERTRU', 'G20PREDBID', 'G20PRELJOR', 'C20PRERTRU', 'C20PREDBID', 'C20PRELJOR', 'G20USSRPER', 'G20USSDOSS', 'G20USSLHAZ', 'S20USSRLOE', 'S20USSRCOL', 'S20USSRGRA', 'S20USSRJAC', 'S20USSRTAY', 'S20USSRJOH', 'S20USSDWAR', 'S20USSDJAC', 'S20USSDLIE', 'S20USSDJOH', 'S20USSDJAM', 'S20USSDSLA', 'S20USSDWIN', 'S20USSDTAR', 'S20USSLSLO', 'S20USSGFOR', 'S20USSIBUC', 'S20USSIBAR', 'S20USSISTO', 'S20USSIGRE', 'G20PSCRSHA', 'G20PSCDBRY', 'G20PSCLMEL', 'G20PSCRMCD', 'G20PSCDBLA', 'G20PSCLWIL', 'R21USSRPER', 'R21USSDOSS', 'R21USSRLOE', 'R21USSDWAR', 'R21PSCRMCD', 'R21PSCDBLA']


### Load Non-Recount Election Data

This data had to be downloaded county-by-county in XML format. The below code parses the XML and grabs the necessary data, adds it to a list, gives it the appropriate column names, and converts the data into a dataframe.

In [94]:
loaded_counties = os.listdir("./raw-from-source/official_county_results/")
z=[]
for locale in loaded_counties:
    print(locale)
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/official_county_results/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                print(lower)
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./County")
                    print(lower_3)
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes,locale])
dfcols = ['state','contest','choice','voting_method','county','num_votes',"file"]
df_general = pd.DataFrame(z,columns=dfcols)

General_Election.xml
[<Element 'Choice' at 0x177810db0>, <Element 'Choice' at 0x1751fcbd0>, <Element 'Choice' at 0x172964a90>]
[<Element 'County' at 0x177810540>, <Element 'County' at 0x177810a90>, <Element 'County' at 0x177810cc0>, <Element 'County' at 0x177810f90>, <Element 'County' at 0x177802860>, <Element 'County' at 0x177802310>, <Element 'County' at 0x177802c70>, <Element 'County' at 0x1778020e0>, <Element 'County' at 0x177802360>, <Element 'County' at 0x177802540>, <Element 'County' at 0x177802720>, <Element 'County' at 0x177802900>, <Element 'County' at 0x177802ae0>, <Element 'County' at 0x177802cc0>, <Element 'County' at 0x177802ea0>, <Element 'County' at 0x177802e50>, <Element 'County' at 0x177802040>, <Element 'County' at 0x1778029a0>, <Element 'County' at 0x177802e00>, <Element 'County' at 0x1778026d0>, <Element 'County' at 0x1778021d0>, <Element 'County' at 0x177802220>, <Element 'County' at 0x177802770>, <Element 'County' at 0x17780bf40>, <Element 'County' at 0x17780b7c0

[<Element 'County' at 0x175537b80>, <Element 'County' at 0x175537c20>, <Element 'County' at 0x175537cc0>, <Element 'County' at 0x175537d60>, <Element 'County' at 0x175537e00>, <Element 'County' at 0x175537ea0>, <Element 'County' at 0x175537f40>, <Element 'County' at 0x17553c040>]
[<Element 'County' at 0x17553c180>, <Element 'County' at 0x17553c220>, <Element 'County' at 0x17553c2c0>, <Element 'County' at 0x17553c360>, <Element 'County' at 0x17553c400>, <Element 'County' at 0x17553c4a0>, <Element 'County' at 0x17553c540>, <Element 'County' at 0x17553c5e0>]
[<Element 'County' at 0x17553c720>, <Element 'County' at 0x17553c7c0>, <Element 'County' at 0x17553c860>, <Element 'County' at 0x17553c900>, <Element 'County' at 0x17553c9a0>, <Element 'County' at 0x17553ca40>, <Element 'County' at 0x17553cae0>, <Element 'County' at 0x17553cb80>]
[<Element 'County' at 0x17553ccc0>, <Element 'County' at 0x17553cd60>, <Element 'County' at 0x17553ce00>, <Element 'County' at 0x17553cea0>, <Element 'County

[<Element 'Choice' at 0x173f0a4a0>, <Element 'Choice' at 0x173f11590>, <Element 'Choice' at 0x16f6c5810>]
[<Element 'County' at 0x173f0a360>, <Element 'County' at 0x173f0a2c0>, <Element 'County' at 0x173f0a220>, <Element 'County' at 0x173f0a180>, <Element 'County' at 0x173f0a0e0>, <Element 'County' at 0x173f0a040>, <Element 'County' at 0x173f08c20>, <Element 'County' at 0x173f08f90>, <Element 'County' at 0x173f08ef0>, <Element 'County' at 0x173f08e50>, <Element 'County' at 0x173f08db0>, <Element 'County' at 0x173f08d10>, <Element 'County' at 0x173f08b80>, <Element 'County' at 0x173f08b30>, <Element 'County' at 0x173f08a90>, <Element 'County' at 0x173f08860>, <Element 'County' at 0x173f088b0>, <Element 'County' at 0x173f08810>, <Element 'County' at 0x173f08770>, <Element 'County' at 0x173f086d0>, <Element 'County' at 0x173f08630>, <Element 'County' at 0x173f08590>, <Element 'County' at 0x173f084f0>, <Element 'County' at 0x173f08450>, <Element 'County' at 0x173f083b0>, <Element 'County' 

In [95]:
df_general.drop(["state","file"], inplace = True, axis = 1)

In [96]:
races_list = ['US House District 1',
       'US House District 2', 'US House District 3',
       'US House District 4', 'US House District 5',
       'US House District 6', 'US House District 7',
       'US House District 8', 'US House District 9',
       'US House District 10', 'US House District 11',
       'US House District 12', 'US House District 13',
       'US House District 14', 'State Senate District 1',
       'State Senate District 2', 'State Senate District 3',
       'State Senate District 4',
       'State Senate Dist 5/Senador Estatal Dist 5',
       'State Senate District 6', 'State Senate District 7',
       'State Senate District 8',
       'State Senate Dist 9/Senador Estatal Dist 9',
       'State Senate District 10', 'State Senate District 11',
       'State Senate District 12', 'State Senate District 13',
       'State Senate District 14', 'State Senate District 15',
       'State Senate District 16', 'State Senate District 17',
       'State Senate District 18', 'State Senate District 19',
       'State Senate District 20', 'State Senate District 21',
       'State Senate District 22', 'State Senate District 23',
       'State Senate District 24', 'State Senate District 25',
       'State Senate District 26', 'State Senate District 27',
       'State Senate District 28', 'State Senate District 29',
       'State Senate District 30', 'State Senate District 31',
       'State Senate District 32', 'State Senate District 33',
       'State Senate District 34', 'State Senate District 35',
       'State Senate District 36', 'State Senate District 37',
       'State Senate District 38',
       'State Senate District 39 - Special Democratic Primary',
       'State Senate District 40', 'State Senate District 41',
       'State Senate District 42', 'State Senate District 43',
       'State Senate District 44',
       'State Senate Dist 45/Senador Estatal Dist 45',
       'State Senate District 46', 'State Senate District 47',
       'State Senate District 48', 'State Senate District 49',
       'State Senate District 50', 'State Senate District 51',
       'State Senate District 52', 'State Senate District 53',
       'State Senate District 54', 'State Senate District 55',
       'State Senate District 56', 'State House District 1',
       'State House District 2', 'State House District 3',
       'State House District 4', 'State House District 5',
       'State House District 6', 'State House District 7',
       'State House District 8', 'State House District 9',
       'State House District 10', 'State House District 11',
       'State House District 12', 'State House District 13',
       'State House District 14', 'State House District 15',
       'State House District 16', 'State House District 17',
       'State House District 18', 'State House District 19',
       'State House District 20', 'State House District 21',
       'State House District 22', 'State House District 23',
       'State House District 24', 'State House District 25',
       'State House District 26', 'State House District 27',
       'State House District 28', 'State House District 29',
       'State House District 30', 'State House District 31',
       'State House District 32', 'State House District 33',
       'State House District 34', 'State House District 35',
       'State House District 36', 'State House District 37',
       'State House District 38', 'State House District 39',
       'State House District 40', 'State House District 41',
       'State House District 42', 'State House District 43',
       'State House District 44', 'State House District 45',
       'State House District 46', 'State House District 47',
       'State House District 48', 'State House District 49',
       'State House District 50', 'State House District 51',
       'State House District 52', 'State House District 53',
       'State House District 54', 'State House District 55',
       'State House District 56', 'State House District 57',
       'State House District 58', 'State House District 59',
       'State House District 60', 'State House District 61',
       'State House District 62', 'State House District 63',
       'State House District 64', 'State House District 65',
       'State House District 66', 'State House District 67',
       'State House District 68', 'State House District 69',
       'State House District 70', 'State House District 71',
       'State House District 72', 'State House District 73',
       'State House District 74', 'State House District 75',
       'State House District 76', 'State House District 77',
       'State House District 78', 'State House District 79',
       'State House District 80', 'State House District 81',
       'State House District 82', 'State House District 83',
       'State House District 84', 'State House District 85',
       'State House District 86', 'State House District 87',
       'State House District 88', 'State House District 89',
       'State House District 90', 'State House District 91',
       'State House District 92', 'State House District 93',
       'State House District 94', 'State House District 95',
       'State House Dist 96/Estatal ante la Asamblea General Dist 96',
       'State House Dist 97/Estatal ante la Asamblea General Dist 97',
       'State House Dist 98/Estatal ante la Asamblea General Dist 98',
       'State House Dist 99/Estatal ante la Asamblea General Dist 99',
       'State House Dist 100/Estatal ante la Asamblea General Dist 100',
       'State House Dist 101/Estatal ante la Asamblea General Dist 101',
       'State House Dist 102/Estatal ante la Asamblea General Dist 102',
       'State House Dist 103/Estatal ante la Asamblea General Dist 103',
       'State House Dist 104/Estatal ante la Asamblea General Dist 104',
       'State House Dist 105/Estatal ante la Asamblea General Dist 105',
       'State House Dist 106/Estatal ante la Asamblea General Dist 106',
       'State House Dist 107/Estatal ante la Asamblea General Dist 107',
       'State House Dist 108/Estatal ante la Asamblea General Dist 108',
       'State House District 109', 'State House District 110',
       'State House District 111', 'State House District 112',
       'State House District 113', 'State House District 114',
       'State House District 115', 'State House District 116',
       'State House District 117', 'State House District 118',
       'State House District 119', 'State House District 120',
       'State House District 121', 'State House District 122',
       'State House District 123', 'State House District 124',
       'State House District 125', 'State House District 126',
       'State House District 127', 'State House District 128',
       'State House District 129', 'State House District 130',
       'State House District 131', 'State House District 132',
       'State House District 133', 'State House District 134',
       'State House District 135', 'State House District 136',
       'State House District 137', 'State House District 138',
       'State House District 139', 'State House District 140',
       'State House District 141', 'State House District 142',
       'State House District 143', 'State House District 144',
       'State House District 145', 'State House District 146',
       'State House District 147', 'State House District 148',
       'State House District 149', 'State House District 150',
       'State House District 151', 'State House District 152',
       'State House District 153', 'State House District 154',
       'State House District 155', 'State House District 156',
       'State House District 157', 'State House District 158',
       'State House District 159', 'State House District 160',
       'State House District 161', 'State House District 162',
       'State House District 163', 'State House District 164',
       'State House District 165', 'State House District 166',
       'State House District 167', 'State House District 168',
       'State House District 169', 'State House District 170',
       'State House District 171', 'State House District 172',
       'State House District 173', 'State House District 174',
       'State House District 175', 'State House District 176',
       'State House District 177', 'State House District 178',
       'State House District 179', 'State House District 180','Constitutional Amendment #1', 'Constitutional Amendment #2',
       'Statewide Referendum A']

In [97]:
df_general = df_general[df_general["contest"].isin(races_list)]

In [98]:
df_general

,contest,choice,voting_method,county,num_votes
20352,US House District 1,"Earl L. """"Buddy"""" Carter (I) (Rep)",Election Day Votes,Bacon,418
20353,US House District 1,"Earl L. """"Buddy"""" Carter (I) (Rep)",Election Day Votes,Brantley,1903
20354,US House District 1,"Earl L. """"Buddy"""" Carter (I) (Rep)",Election Day Votes,Bryan,2635
20355,US House District 1,"Earl L. """"Buddy"""" Carter (I) (Rep)",Election Day Votes,Camden,3561
20356,US House District 1,"Earl L. """"Buddy"""" Carter (I) (Rep)",Election Day Votes,Charlton,1091
...,...,...,...,...,...
29815,Statewide Referendum A,NO,Provisional Votes,Whitfield,19
29816,Statewide Referendum A,NO,Provisional Votes,Wilcox,0
29817,Statewide Referendum A,NO,Provisional Votes,Wilkes,2
29818,Statewide Referendum A,NO,Provisional Votes,Wilkinson,2


In [99]:
fields_dict = {}
for val in os.listdir("./cand_dicts/"):
    holder = pd.read_csv("./cand_dicts/"+val)
    holder_dict = dict(zip(holder["Cand_Name"], holder["Field_Name"]))
    print(len(holder_dict))
    fields_dict.update(holder_dict)

87
38
28
267


In [100]:
fields_dict

{'Ben Watson (I) (Rep)': 'GSU01RWAT',
 'Lester G. Jackson, III (I) (Dem)': 'GSU02DJAC',
 'Sheila McNeill (Rep)': 'GSU03RMCN',
 'Billy Hickman (I) (Rep)': 'GSU04RHIC',
 'Sheikh Rahman (I) (Dem)': 'GSU05DRAH',
 'Jennifer ""Jen"" Jordan (I) (Dem)': 'GSU06DJOR',
 'Harrison Lance (Rep)': 'GSU06RLAN',
 'Tyler Harper (I) (Rep)': 'GSU07RHAR',
 'Treva Gear (Dem)': 'GSU08DGEA',
 'Russ Goodman (Rep)': 'GSU08RGOO',
 'Nikki Merritt (Dem)': 'GSU09DMER',
 'P. K. Martin, IV (I) (Rep)': 'GSU09RMAR',
 'Emanuel Jones (I) (Dem)': 'GSU10DJON',
 'Dean Burke (I) (Rep)': 'GSU11RBUR',
 'Freddie Powell Sims (I) (Dem)': 'GSU12DSIM',
 'Tracy Taylor (Rep)': 'GSU12RTAY',
 'Mary Egler (Dem)': 'GSU13DEGL',
 'Carden H. Summers (I) (Rep)': 'GSU13RSUM',
 'Travis Johnson (Dem)': 'GSU14DJOH',
 'Bruce Thompson (I) (Rep)': 'GSU14RTHO',
 'Ed Harbison (I) (Dem)': 'GSU15DHAR',
 'Cinquez Jester (Dem)': 'GSU16DJES',
 'Marty Harbin (I) (Rep)': 'GSU16RHAR',
 'Kelly Rose (Dem)': 'GSU17DROS',
 'Brian Strickland (I) (Rep)': 'GSU17RST

In [111]:
df_general["choice"] = np.where(df_general["contest"].isin(["Statewide Referendum A",'Constitutional Amendment #1', 'Constitutional Amendment #2']), df_general["contest"]+df_general["choice"], df_general["choice"])

In [112]:
df_general["choice"] = df_general["choice"].str.strip()

In [118]:
cand_update_dict = {'Jill  Prouty (Dem)':'Jill Prouty (Dem)', 
                   'Constitutional Amendment #1YES':'Amendment #1 "Yes"',
                    'Constitutional Amendment #1NO':'Amendment #1 "No"',
                   'Constitutional Amendment #2YES':'Amendment #2 "Yes"',
                    'Constitutional Amendment #2NO':'Amendment #2 "No"',
                   'Statewide Referendum AYES':'Referendum A "Yes"',
                    'Statewide Referendum ANO':'Referendum A "No"'}

In [119]:
df_general["choice"] = df_general["choice"].map(cand_update_dict).fillna(df_general["choice"])
# df_general["choice"] = df_general["choice"].str.replace("  ", " ")

In [120]:
df_general["Clean_Cand"] = df_general["choice"].map(fields_dict).fillna("n/a")

In [121]:
df_general["Clean_Cand"].value_counts()

G20RFANO      636
G20A02NO      636
G20A01NO      636
G20A02YES     636
G20A01YES     636
             ... 
GSL057DEVA      4
GSL058DCAN      4
GSL059DDRE      4
GSL074DNEA      4
GSL047DCAR      4
Name: Clean_Cand, Length: 388, dtype: int64

In [132]:
df_general["num_votes"] = df_general["num_votes"].astype(int)

In [133]:
df_general_pivoted = pd.pivot_table(df_general, index = "county", columns = "Clean_Cand", values = "num_votes", aggfunc = sum)

In [135]:
df_general_pivoted.reset_index(inplace = True, drop = False)

In [137]:
df_general_pivoted = df_general_pivoted.fillna(0)

In [141]:
for val in list(df_general_pivoted.columns):
    if val != "county":
        df_general_pivoted[val] = df_general_pivoted[val].astype(int)

In [143]:
df_general_pivoted.to_csv("./cleaned_official_county_totals.csv", index = False)